In [1]:
import pandas as pd
import numpy as np
import os
import itertools
from itertools import product
import math

sale = pd.read_excel('1402_sales.xlsx')
TARGET = pd.read_excel('target_daya.xlsx')
center_factors=pd.read_excel('center_factors.xlsx')
mapping = pd.read_excel('ProdMapping2.xlsx') 

In [2]:
sale['ProductNameFA']=sale.ProductName
replace_dict = dict(zip(mapping['DistProductName'],mapping['ProductNameFA']))
sale['ProductNameFA'].replace(replace_dict,inplace=True)
sale['month'] = sale['SDate'].apply(lambda t: int(str(t)[4:6]))

In [3]:
TARGET = TARGET.assign(key=1)
center_factors = center_factors.assign(key=1)
merged_df = TARGET.merge(center_factors, on='key').drop(columns='key')

# Calculate the target for each center based on the distribution factor
merged_df['Target'] = merged_df['Target'] * (merged_df['factor'] )

# Drop unnecessary columns
#merged_df = merged_df.drop(columns=['Supply', 'SupplyRiali' ,'TargetRiali' , 'Stock ' , 'factor_compl'])
target_final  = pd.DataFrame(merged_df)
# Display the resulting DataFrame
target_final.to_excel('target_final.xlsx' , index =False)

In [4]:
sales = sale.groupby(['ProductNameFA','DistributionCenter' ,'month'])['NetSalesVolumePack'].sum()
sales = pd.DataFrame(sales)
sales.reset_index(inplace=True)
sales.to_excel('sales.xlsx')

In [5]:
# Columns to join on
columns_to_join_on = ['ProductNameFA', 'DistributionCenter' , 'month']

# Perform an outer join using merge
target_Sales = pd.merge(target_final, sales, on=columns_to_join_on, how='outer')

# Fill NaN values with 0
target_Sales['NetSalesVolumePack'].fillna(0, inplace=True)
target_Sales['Target'].fillna(0, inplace=True)

In [6]:
target_Sales.to_excel('target_Sales_1402.xlsx', index=False)